# Merging data

Merging the data can be done through different situations:

### Merging similar datasets 

If the attribute of the input datasets are exactly the same using the pandas library they can be merged by connecting to each other vertically as the following lines.

First the libraries of pandas and geopandas are imported.

In [1]:
from pathlib import Path
import geopandas as gp
import pandas as pd

In [2]:
INPUT = Path.cwd().parents[0] / "00_data"
OUTPUT = Path.cwd().parents[0] / "out"

Then the path to the input datasets are defined.

In [4]:
layers_path = Path(INPUT / "layers")
input_layer = gp.read_file(layers_path / "border.shp")
second_layer = gp.read_file(layers_path / "border.shp")

Then using the function **concat**, refers to concatenate, the two layers are connected to each other, the ignore_index parameter will set to `True` to have the new index for the features in the merged output layer. 

```{danger}
If the ignore_index parameter change to `False`, then the index from the original layers will be preserved. In this case, if these is some similar indexes in two datasets, will cause the duplicate indexes in the output layer. 
```

In [5]:
merged_layer = pd.concat([input_layer, second_layer], ignore_index=True)

The result will be printed:

In [6]:
merged_layer

,Id,geometry
0,0,"POLYGON ((405728.211 5659356.174, 405708.836 5..."
1,0,"POLYGON ((405728.211 5659356.174, 405708.836 5..."


### Merging based on common attributes

If there is a common attribute in both datasets they can merge in one dataset. In this case the output will include just the rows (features) that are exactly the same in the common attribute.

**ex:** Considering attribute `ID` as the common attribute, just the rows including the **same ID** will horizontally join, meaning that all the attributes for that ID, from the both datasets, will be included in the output.  

For achieving the package geopandas imported:

Then the path for the two datasets are defined:

In [7]:
input_layer = gp.read_file(layers_path / "border.shp")
second_layer = gp.read_file(layers_path / "border.shp")

Then using the merge function the two datasets are connected horizontally, In the merge function the parameter `on` defines the similar attribute.

The how parameter can get the different values:
* `inner`: Result includes only the rows from both datasets that have the same value in common attribute. (ex: Here same Id)
* `outer`: Result includes all the rows of two datasets. For the rows without match values in the other dataset (ex: Here match Id ), missing values fill with **NaN**. 
* `left`: Result includes all the rows from the first layer that call the merge function and the matched data from the other layer. For the rows without match values in the other dataset (ex: Here match Id ), missing values fill with **NaN**.
* `right`: Result includes all the rows from the second layer that is defined through the merge function and the matched data from the other layer. For the rows without match values in the other dataset (ex: Here match Id ), missing values fill with **NaN**.

**Example:**

`Dataset 1`:

|ID  |Type  |
|:---| ----:|
|1   |     A|
|3   |C     |
|6   |D     |


`Dataset 2`:

|    ID  |   Time    |
| :------| --------: |
|    1   |  15 min   |
|    3   |  16 min   |
|    4   |  17 min   |


- inner

|    ID  |  Type   |   Time   |
| :------| ------- |--------: |
|    1   |   A     |  15 min  |
|    3   |   C     |  16 min  |

- outer

|    ID  |  Type   |   Time   |
| :------| ------- |--------: |
|    1   |   A     |  15 min  |
|    3   |   C     |  16 min  |
|    6   |   D     |  **NaN**     |
|    4   |   **NaN**   |  17 min  |

- left

|    ID  |  Type   |   Time   |
| :------| ------- |--------: |
|    1   |   A     |  15 min  |
|    3   |   C     |  16 min  |
|    6   |   D     | **NaN**  |

- right

|    ID  |  Type   |   Time   |
| :------| ------- |--------: |
|    1   |   A     |  15 min  |
|    3   |   C     |  16 min  |
|    4   |   **NaN**   |  17 min  |

In [8]:
merged_layer = input_layer.merge(second_layer, on='Id', how='inner')

Then the result will be checked using the print function:

In [9]:
merged_layer

,Id,geometry_x,geometry_y
0,0,"POLYGON ((405728.211 5659356.174, 405708.836 5...","POLYGON ((405728.211 5659356.174, 405708.836 5..."


```{note}
In case there are other attribute (column) than the similar attribute that was the base of the merge function, by default the Geopandas will add the suffixes `_x` and `_y` to the name of the attributes. 
**ex:** The merge function between two polygon are done based on the similar "ID". As the polygons have the attribute "area", the merged layer will have two column of "area":
`area_x` and `area_y`. 
Coustomizing these suffixes explained below.
```

- Customizing suffixes

Changing the name of the attributes is done using the `suffixes` parameter.

In the parameter `suffixes` of the merge function, the first value is the suffix for the input layer, and the second value is the suffix of the second layer. 

In [10]:
merged_layer = input_layer.merge(
    second_layer,
    on='Id',
    how='inner',
    suffixes=('_inputlayer', '_2ndlayer'))

Then the result will be checked using the print function:

In [13]:
merged_layer.T

,0
Id,0
geometry_inputlayer,"POLYGON ((405728.2110448944 5659356.17364894, ..."
geometry_2ndlayer,"POLYGON ((405728.2110448944 5659356.17364894, ..."


By using the name of the layers directly in the merge function, all the attributes from  both layers will be included in the merged layer,it is also possible to define required attributes from each layer.

For defining the specific attributes for merging process, these attributes are defined in the double brackets `[[`. The "common" attribute (`on` parameter), **must** be defined in both layers. 

```{warning} 
The single bracket `[` makes a series of data which is not in tabular format (GeoDataFrame) and makes an error with operating the merge function. The double brackets `[[` save the tabular structure of the data.
```

In the following line, just the "Id" which is the common attribute between two layers and "geometry" attribute from the second layer included in the merged layer.

In [23]:
merged_layer = input_layer[['Id']].merge(
    second_layer[['Id','geometry']],
    on='Id',
    how='inner')
merged_layer

,Id,geometry
0,0,"POLYGON ((405728.211 5659356.174, 405708.836 5..."


### Spatial join

Another way of merging data is based on the spatial relationship between datasets.

For this reason first the geopandas package imported:

Then the interested layers are loaded:

In [25]:
input_layer = gp.read_file (OUTPUT / "clipped.shp")
join_layer = gp.read_file(INPUT / "layers" / "border.shp")

Then the "sjoin" method is used for this goal. This method has two important parameters such as `predicate`, `how`. 

The `predicate` parameter in sjoin method defines the type of spatial relationship:
* **`contains`**: Object A **completely** encloses object B, no boundary touch.
* **`covers`**: Object A completely contains object B, boundaries touch.
* **`within`**: Object A is **completely** inside object B, no boundary touch.
* **`covered_by`**: Object A is completely within object B, boundaries touch.
* **`touches`**: Objects A and B meet **only** at boundaries.
* **`overlaps`**: Objects A and B share area.
* **`crosses`**: Objects A and B contact at points.
* **`intersects`**: Object A and B touch or cross or share area.

```{tip} 
Visit [here](https://desktop.arcgis.com/en/arcmap/latest/extensions/data-reviewer/types-of-spatial-relationships-that-can-be-validated.htm#GUID-B8BCA279-A7D9-422D-90B6-414B11350D1A) for getting more familiar with spatial relationships. 
```

```{figure} ../resources/13.png
:height: 150px
:name: figure-example

Spatial relationships
```

The `how` parameter, as mentioned in the previous section, specifies how rows from two layers are combined after spatial join. 

At the end the layers are defined in the method "sjoin" and the spatial join will be done. The output includes all the columns from both layers and an "index column" that shows the index of the join feature.

**ex:** If it is a **right join** the index will be from the left dataset **index_left** and if it is a **left join** the index will be from the right dataset **index_right**. In the **inner join** also the index will be from the right dataset **index_right**.

In [26]:
intersected = input_layer.sjoin(join_layer, predicate='intersects', how='left')

The result printed using the print function.

In [28]:
list(intersected.columns)

['KS_IS',
 'CLC_st1',
 'CLC18',
 'CLC',
 'Biotpkt201',
 'Shape_Leng',
 'Shape_Area',
 'geometry',
 'index_right',
 'Id']

In [30]:
intersected.head(3)

,KS_IS,CLC_st1,CLC18,CLC,Biotpkt201,Shape_Leng,Shape_Area,geometry,index_right,Id
0,SV,122,None,NaN,5.271487,487783.286284,2.869516e+06,"POLYGON ((401808.569 5661532.707, 401859.892 5...",0,0
1,None,231,231,231.0,10.981298,1385.654150,1.235155e+04,"POLYGON ((403104.213 5657996.304, 403105.003 5...",0,0
2,None,231,231,231.0,10.981298,2978.763179,4.237039e+04,"POLYGON ((403076.477 5658033.218, 403036.719 5...",0,0


These suffixes are the default names for the indexes which are also customizable. Using the `lsuffix` parameter for the input layer (**l**eft GeoDataFrame ) and `rsuffix` parameter for the second layer (**r**ight GeoDataFrame), the suffixes customize separately. It is just needed to define the interested suffixes in the `lsuffix` or `rsuffix` parameters

In the following example as the type of join is "inner" and **by default** the values from the second layer will be join to the first layer, the indexes will be from the second layer (right layer), So the "rsuffix" customized.

In [32]:
intersects_result = input_layer.sjoin(
    join_layer, predicate='intersects', how='inner', rsuffix='_border')
list(intersects_result.columns)

['KS_IS',
 'CLC_st1',
 'CLC18',
 'CLC',
 'Biotpkt201',
 'Shape_Leng',
 'Shape_Area',
 'geometry',
 'index__border',
 'Id']

It is also possible to define the interested columns from the available columns of both datasets to have in the spatial join output, these columns are defined in the double brackets `[[`. As it is spatial join and it is done based on the geometry of the features, the `geometry` attribute **must** be defined in both layers. 

In [33]:
intersects_result = input_layer[['KS_IS', 'CLC','geometry']] \
    .sjoin(
        join_layer[['Id','geometry']],
        predicate='intersects', how='right')

list(intersects_result.columns)

['index_left', 'KS_IS', 'CLC', 'Id', 'geometry']

The second way of spatial join is using the `gp.sjoin()` function from geopandas. The parameters are exactly the same as the first method, the only difference is that both datasets are called inside the function, as the functions generally work.

In [34]:
intersects_result= gp.sjoin(
    input_layer, join_layer, how='right',
    predicate='intersects', lsuffix='_left')
list(intersects_result.columns)

['index__left',
 'KS_IS',
 'CLC_st1',
 'CLC18',
 'CLC',
 'Biotpkt201',
 'Shape_Leng',
 'Shape_Area',
 'Id',
 'geometry']

Here also it is possible to define specific columns in the double brackets `[[`  to take part in the operation. As it is spatial join, the `geometry` attribute **must** be defined in both layers.

In [35]:
intersects_result= gp.sjoin(
    input_layer[['KS_IS', 'CLC','geometry']],
    join_layer[['Id','geometry']],
    how='right', predicate='intersects', lsuffix='_left')
list(intersects_result.columns)

['index__left', 'KS_IS', 'CLC', 'Id', 'geometry']